In [62]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
import time
warnings.filterwarnings('ignore')


In [76]:
class Auto_Inviter():
    def __init__(self,user_profile_path, chromedriver_path):
        options = webdriver.ChromeOptions()
        options.add_argument('--user-data-dir=' + user_profile_path) 
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")
        options.add_argument("--disable-extensions")
        self.driver = webdriver.Chrome('./chromedriver98',chrome_options=options)
        self.driver.maximize_window()
    def open_website(self, url, company_name):
        self.driver.get(url)
        self.company = company_name
    def get_user_names(self):
        WebDriverWait(self.driver, 5, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='visually-hidden']")))
        titles = self.driver.find_elements_by_class_name('visually-hidden')
        names = []
        for title in titles:
            if 'View' in title.text:
                name = title.text.replace('\n','').replace(('View '),'').replace(('’s profile'),'')
                names.append(name)
        return names
    def open_connector_pagination(self, target):
        ActionChains(self.driver).key_down(Keys.COMMAND).perform()
        target.click()
        ActionChains(self.driver).key_up(Keys.COMMAND).perform()
        self.driver.switch_to.window(self.driver.window_handles[1])

    def close_connector_pagination(self):
        self.driver.close()
        self.driver.switch_to.window(self.driver.window_handles[0])

    def next_page(self):
        WebDriverWait(self.driver, 10, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='artdeco-button__text' and text()='Next']"))).click()

    def send_msg(self, name, msg):
        msg = 'Hi '+name.split(' ')[0]+',\n' + msg
        if len(msg) > 300:
            print('The message is too long and there will be some missing words!!!')
            print('fail')
            return 1
        self.driver.switch_to.window(self.driver.window_handles[1])
        try:
            self.driver.find_elements_by_xpath('//button[@aria-label="More actions"]')[1].click()
            self.driver.find_elements_by_xpath("//span[@class='display-flex t-normal flex-1' and text()='Connect']")[1].click()
            self.driver.find_elements_by_xpath("//span[@class='artdeco-button__text' and text()='Connect']")[0].click()
        except:
            self.driver.find_elements_by_xpath("//span[@class='artdeco-button__text' and text()='Connect']")[1].click()
        self.driver.find_element_by_xpath('//button[normalize-space()="Add a note"]').click()
        self.driver.find_element_by_id('custom-message').send_keys(msg)
        #self.driver.find_element_by_xpath("//span[@class='artdeco-button__text' and text()='Send']").click()
        print('sucess')
        #time.sleep(3)

    def ask_for_referral(self, msg, num):
        i = 0
        while True:
            try:
                WebDriverWait(self.driver, 5, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='app-aware-link']")))
            except:
                self.driver.find_element_by_xpath("//span[@class='artdeco-button__text' and text()='Retry search']").click()
                print('renew')
                continue
            links = self.driver.find_elements_by_class_name('app-aware-link')
            names = self.get_user_names()
            for link in links:
                name = link.text.split('\n')[0]
                if name in names:
                    names.remove(name)
                    i+=1
                    print(i,": ",name)
                    self.open_connector_pagination(link)
                    self.send_msg(name,msg)
                    self.close_connector_pagination()
                if i == num:
                    break
                    #return
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            self.next_page()

In [78]:
company_url = 'https://www.linkedin.com/search/results/people/?keywords=amazon&origin=GLOBAL_SEARCH_HEADER&sid=wUZ'
company_name = 'google'
num = 30
msg ='My name is Alex, a UCSD MSCS student. I found your profile on Linkedln. Moreover, currently I\'m looking for internship in the USA and I know that you work at '+company_name+' and I\'m impressed by you experience. I\'m just wondering can I get more connection or referral from you.'

user_profile_path = '/Users/yanghangwu/Library/Application Support/Google/Chrome/Profile 3'
chromedriver_path = './chromedriver98'
Inviter = Auto_Inviter(user_profile_path, chromedriver_path)
Inviter.open_website(company_url, company_name)
Inviter.ask_for_referral(msg, num)

1 :  Zabie Wahab
sucess
2 :  Weiduo Sun
sucess
3 :  Jiawen Qi
sucess
4 :  Ming Cai
sucess
5 :  Fan Wu
sucess
6 :  Shih-Chen (Dennis) Lin
sucess
7 :  Rou Zhang


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=98.0.4758.109)
Stacktrace:
0   chromedriver98                      0x0000000102b80b50 chromedriver98 + 4442960
1   chromedriver98                      0x0000000102b15d14 chromedriver98 + 4005140
2   chromedriver98                      0x000000010276a30c chromedriver98 + 156428
3   chromedriver98                      0x000000010274e468 chromedriver98 + 42088
4   chromedriver98                      0x00000001027b48e4 chromedriver98 + 461028
5   chromedriver98                      0x00000001027c2294 chromedriver98 + 516756
6   chromedriver98                      0x000000010278e660 chromedriver98 + 304736
7   chromedriver98                      0x0000000102b43308 chromedriver98 + 4190984
8   chromedriver98                      0x0000000102b59584 chromedriver98 + 4281732
9   chromedriver98                      0x0000000102b5e3d0 chromedriver98 + 4301776